## Build Logistic Regression  

### Import packages 

In [1]:
import pandas as pd
import numpy as np

### Load the preprocessed data 

In [2]:
data_preprocessed = pd.read_csv('Absenteeism-preprocessed.csv')
data_preprocessed.head(3)

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2


### Create targets and inputs

In [3]:
median = data_preprocessed['Absenteeism Time in Hours'].median()

# if absence time is greater than median, than target is 1; otherwise target is 0
# comparing with median can also balance the dataset, i.e. equal number of 1s and 0s
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > median, 1, 0)

data_preprocessed['Excessive Absenteeism'] = targets

In [4]:
data_with_targets = data_preprocessed.drop(labels=['Absenteeism Time in Hours', 'Day of the Week',
                                                   'Distance to Work', 'Daily Work Load Average'], axis=1)

In [5]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

In [6]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

### Standardize inputs

Only standardize numerical values, while leave dummy variables untouched

In [7]:
# Create a custom scaler that can standardize certain columns of a dataframe
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScalar(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns, copy=True, with_mean=True, with_std=True):
        self.scaler = StandardScaler(copy, with_mean, with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns]) 
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_unscaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_scaled, X_unscaled], axis=1)[init_col_order]

In [8]:
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Education']
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [9]:
absenteeism_scaler = CustomScalar(columns_to_scale)

In [10]:
absenteeism_scaler.fit(unscaled_inputs)

CustomScalar(columns=['Month Value', 'Transportation Expense', 'Age', 'Body Mass Index', 'Children', 'Pets'],
       copy=None, with_mean=None, with_std=None)

In [11]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

scaled_inputs.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487


### Split the data into train & test and shuffle

In [12]:
from sklearn.model_selection import train_test_split

# train_test_split shuffle the dataset automatically, fixed random state guarantees every shuffle gets same result
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size=0.8, random_state=20)

D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Logistic regression with sklearn 

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [14]:
reg = LogisticRegression()

reg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
print("Train accuracy: {0:f}%".format(reg.score(x_train, y_train)*100))

Train accuracy: 77.500000%


### Create summary table

1. Since features are standardized, larger the coefficients, more important the feauture is. 
2. If coefficient of a feature is close zero, this means the feature is not important.

In [16]:
feature_name = unscaled_inputs.columns.values

summary_table = pd.DataFrame(feature_name, columns=['Feature name'])
summary_table['Coefficient'] = reg.coef_.T

In [17]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()

summary_table.sort_values('Coefficient', ascending=False)

,Feature name,Coefficient
3,Reason_3,2.960507
1,Reason_1,2.627499
2,Reason_2,0.863386
4,Reason_4,0.663907
6,Transportation Expense,0.599798
10,Children,0.342497
8,Body Mass Index,0.275685
5,Month Value,0.154937
7,Age,-0.172451
9,Education,-0.234525


###  Test the model

In [18]:
print("Test accuracy: {0:f}%".format(reg.score(x_test, y_test)*100))

Test accuracy: 75.000000%


## Save the model 

Saving the model is equivalent to saving the LogisticRegression() object as it contains all the information (intercept, coefficients etc) about the model

In [22]:
# pickle is a Python module that converts a Python object into a character stream
import pickle

In [23]:
# 'model' is file name, 'wb' is write byte which means we are writing contents in the file
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [29]:
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)